## Image Classification
A quick example on how to use the visual recognition API to classify images

In [ ]:
# Install the API
!pip install --upgrade "watson-developer-cloud>=1.4.0"

### API key
Make sure to change the API key to the value from your own service

In [ ]:
# Replace the value of the credentials variable with the credentials from bluemix.net

credentials = {
  "apikey": "43t1dCi_vk1A_X6mm5kcUNdNCryTM_YZAqvxNJqja7QW",
  "iam_apikey_description": "Auto generated apikey during resource-key operation for Instance - crn:v1:bluemix:public:watson-vision-combined:us-south:a/e46675b7f1bf89b09b5badfb3bd4a7b5:0689e33a-867c-433a-bd2a-d8c02319af7b::",
  "iam_apikey_name": "auto-generated-apikey-65263c4f-9eb6-4d5f-9737-fc89ac51f283",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/e46675b7f1bf89b09b5badfb3bd4a7b5::serviceid:ServiceId-b9150d5b-c9ab-4806-80b3-6685f4dce204",
  "url": "https://gateway.watsonplatform.net/visual-recognition/api"
}

In [ ]:
# Create the interface object
import json
from watson_developer_cloud import VisualRecognitionV3

visual_recognition = VisualRecognitionV3(
    version='2018-03-19',
    url=credentials['url'],
    iam_apikey=credentials['apikey']
)

## Access local files
The local files are stored in the IBM Cloud Object Storage.<br/>
IBM COS supports a common set of S3 API functions for programmatic access. Watson Studio can do most of the work of defining the access to your files.

Use the empty cell below to insert code, let's say from "elantra.jpg", using the "Insert to code" button and the "Insert StreamingBody object".

Compare the code to the next cell and modify the next cell:<br/>
 - Replace the four lines starting at `ibm_api_key_id` with the ones you just created
 - Copy the value of Bucket in the client_xxx*.getobject() call and replace the value of bucket in the next cell

In [ ]:
import sys
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='_4IpLk7tjLaIoqYHLlUlzutjpTXtGZK1II8pfM2HmiWc',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

Bucket='tutorial2-donotdelete-pr-7vkp9vgdxzvssm'

In [ ]:
# Create a zip files with the three images we want to classify
res=cos.download_file(Bucket=Bucket,Key='orange-bmw-m3-coupe.jpg',Filename='orange-bmw-m3-coupe.jpg')
res=cos.download_file(Bucket=Bucket,Key='volvo-x90-suv.jpg',Filename='volvo-x90-suv.jpg')
res=cos.download_file(Bucket=Bucket,Key='elantra.jpg',Filename='elantra.jpg')
!zip cars.zip *.jpg

## Calling the classifier
First, replace the classifier ID with the ID of your classifier

You will execute the next two cells twice:
- With the classifier_ids parameter commented out.<br/>
This will show results with the generic classifier that is part of the service
- Uncomment the classifier_ids and execute again<br/>
This time, you get the results from your classifier

Note that both classifier ('default' and your own) can be used at the same time.

In [ ]:
# Classify the images and print out partial results for readability
car_1=open("./cars.zip", 'rb')
classes = visual_recognition.classify(
        car_1,
        parameters=json.dumps({
            # 'classifier_ids': ['CarClassifier_973954551'],
            'threshold': 0.1
        })).get_result()
# print(json.dumps(classes, indent=2))
# Format the json output to be easier to read
for i in range(3):
    result = classes['images'][i]
    print ("Image: " + result['image'])
    print ("Class: " + str(result['classifiers'][0]['classes'][0]['class']))
    print ("Score: " + str(result['classifiers'][0]['classes'][0]['score']))
    print ("Classifier: " + str(result['classifiers'][0]['classifier_id']))
    print(" ")

In [ ]:
# Print out the entire json file
print(json.dumps(classes, indent=2))

In [ ]:
# remove the local files create earlier
!rm *.zip *.jpg